# NN and LSTM Beginners version

**Audience:**
This version of the workshop is dedicated to participants with limited knowledge about neural networks, Keras and Python.

**Purpose:**
Provide participants with some examples of how to use neural network for time series problem. Participant is encouraged to experiment with architectures to see how that influences the results.

The following helper functions are provided:
* load_data - load the data from the csv file
* create_dataset - creates two numpy arrays with target and features. Features are lagged prices of the stock
* out_of_time - split in train and test

Overview of the tutorial:

* Data Praparetion:
  - load the data 
  - explore the target
  - convert data to numpy array
  - make features: lagged price
  - split the data intro train and test 
  - normalize the data

* Feed Forward Neural Network:
    - Define the neural network input shape
    - Define the number of neurons, layers and activation functions
    - Define the output of the neural network
    - Define the loss of the neural network
    - Compile the neural network
    - Train the neural network
    - Keep on eye on the loss, is the model training?
    - Plot the predictions again the real data
    - Redo the above with different paraments and model architectures
    
* LSTM:
    - Define the neural network input shape. Reminder: LSTM needs a special shape
    - Define the number of neurons, layers and activation functions
    - Define the output of the neural network
    - Define the loss of the neural network
    - Compile the neural network
    - Train the neural network
    - Keep on eye on the loss, is the model training?
    - Plot the predictions again the real data
    - Redo the above with different paraments and model architectures
    
    
Neural Networks Cheat Sheet: https://stanford.edu/~shervine/teaching/cs-229/cheatsheet-deep-learning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

## Helper Functions

In [ ]:
def load_data(symbol = 'SP500'):
    data = pd.read_csv('data_stocks.csv')
    data = data[['DATE',symbol]].sort_values(by=['DATE'])
    return data

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

def out_of_time(dataset, train_size):
    train_size = int(len(dataset) * train_size)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    return train, test

scaler = MinMaxScaler(feature_range=(0, 1))

## Feed Forward NN - Data Preparation

In [ ]:
# loading the data, by default we use S&P500
stock_data = load_data()

# visualising the data
stock_data['SP500'].plot(figsize = (15,10))

# connverting data to numoy arrays. 
stock_data = stock_data['SP500'].values.reshape(stock_data.shape[0],1)

# normalizing the data. Remember, neural nets work better with normalized data
stock_data = scaler.fit_transform(stock_data)

# Split by trian and test. First 85% of the data for training and the rest is for
# testing
train, test = out_of_time(stock_data,0.85)

# using lagged price of the stock as feature. Here look back is number of
# previous prices to take as a feature
X_train, Y_train = create_dataset(train, look_back=5)
X_test, Y_test = create_dataset(test, look_back=5)

## Feed Forward NN - Bulding the Network

Here we define the model.

We will do a regular feedworward model, so we select model type Sequential

In [ ]:
model = Sequential()

Now we need to define the first hidden layer in the neural network.
We specify the shape of the input to the hidder layer and the shape of the hidden layer.

We have 5 features, thus shape of our input to the naural network is 5.
We want 10 neurons in the hidder layer, so we specify dinemtion of 10.
In addition we apply a relu activation funtion to the output of first hidden layer

In [ ]:
model.add(Dense(10, input_dim=5, activation='relu'))

To add a second layer we simply use add method of the model here we only need to speciy the dimentions and activation function

In [ ]:
model.add(Dense(20, activation='relu'))

Now we need to specify the output layer of the neural network. since it is a regression problem, out output is one number and shape is 1. Also, no activation function is spplied, by using 'linear' option

In [ ]:
model.add(Dense(1, activation='linear'))

Now we have our architecture figured out, we need to define loss and which algorythim to use for training the model weights. Since it is a regression problem we use mean square error MSE, with adam optimizer


In [ ]:
model.compile(loss='mse', optimizer='adam')

Now we simply need to fit the model on the data. Here we need to specify the number of epochs, that is the number of time neural network will go through the dataset to train

While model is training, keep an eye on the loss, is it decreasing?

In [ ]:
history = model.fit(X_train,Y_train,epochs=4,validation_data=(X_test,Y_test),shuffle=False)

## Feed Forward NN - Visualizing the results

In [ ]:
# Now we will plot the prediction of the model versus actual data to see how well it trained

# First we need to tranform the data back to original scare
prediction_nn = scaler.inverse_transform(model.predict(X_test))

# Then we need to reshape the data into the correct dimention
prediction_nn = prediction_nn.reshape(prediction_nn.shape[0], )

# Also we need to transform the target to the ofiginal scale
true_data = scaler.inverse_transform([Y_test])

# And now we can plot the results
plt.plot(prediction_nn)
plt.plot(true_data[0])

## Feed Forward Neural Network - Self Activities

 * change the number of features
 * add more layers/neurons
 * try different activation functions
 * was scaling done correct? we scaled before splitting the data

## LSTM - Data Preparation

In [ ]:
# loading the data, by default we use S&P500
stock_data = load_data()

# visualising the data
stock_data['SP500'].plot(figsize = (15,10))

# connverting data to numoy arrays. 
stock_data = stock_data['SP500'].values.reshape(stock_data.shape[0],1)

# normalizing the data. Remember, neural nets work better with normalized data
stock_data = scaler.fit_transform(stock_data)

# Split by trian and test. First 85% of the data for training and the rest is for
# testing
train, test = out_of_time(stock_data,0.85)

# using lagged price of the stock as feature. Here look back is number of
# previous prices to take as a feature
X_train, Y_train = create_dataset(train, look_back=5)
X_test, Y_test = create_dataset(test, look_back=5)

So far the steps of the data preparation are exactly the same as in the previous part. However, LSTMs require data to be in specific shape. 

LSTMs require data to be a 3D array with following elements:
 * length training data -> N
 * length of sequance we want to predict -> Y_l
 * number of features -> X_l
 
So the 3D array takes the following form: (N, Y_l, X_l)

In our case, the length of the training data is 35070, we are predicting sequance of 1 (next price) and we have 5 features. 

Thus our train data has shape: (35070, 1 , 5). Same is done for test data.

I do advise to look into the final data and see how it looks like.

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

## LSTM - Bulding the Network

Like before, we first initiate a model type. Here it is again sequential model

In [ ]:
model = Sequential()

Now we add an LSTM cell to our model. We need to specify model input shape and number of neurons on the LSMT cell. 

We will be feeding one observat at a time with 5 features, so our input shape is: (1,5)

LSTM sells are rather standardised in terms of activations functions and achitecture inside the cell. We can change number of neurons in the cell. That is done by specifying 10 as shape for the cell. 

In princical you can shange the entire structure of the LSTM cell, like activation functions and such, but for that you need to use Tensor Flow.

In [ ]:
model.add(LSTM(10, input_shape = (1, X_train[0].shape[1])))

Now we need to specify the output layer of the neural network. since it is a regression problem, out output is one number and shape is 1. Also, no activation function is spplied, by using 'linear' option

In [ ]:
model.add(Dense(1, activation='linear'))

Now we have our architecture figured out, we need to define loss and which algorythim to use for training the model weights. Since it is a regression problem we use mean square error MSE, with adam optimizer

In [ ]:
model.compile(optimizer='adam',loss='mse')

Now we simply need to fit the model on the data. Here we need to specify the number of epochs, that is the number of time neural network will go through the dataset to train. Keep in mind that LSTMs have a lot of parameters, so in genral they need a lot epochs to train properly.

While model is training, keep an eye on the loss, is it decreasing?

In [ ]:
history = model.fit(X_train,Y_train,epochs=15,validation_data=(X_test,Y_test),shuffle=False)

## LSTM - Visualizing the results

In [ ]:
# Now we will plot the prediction of the model versus actual data to see how well it trained

# First we need to tranform the data back to original scare
prediction_lstm = scaler.inverse_transform(model.predict(X_test))

# Then we need to reshape the data into the correct dimention
prediction_lstm = prediction_lstm.reshape(prediction_lstm.shape[0], )

# And now we can plot the results
plt.plot(prediction_nn)
plt.plot(prediction_lstm)
true_data = scaler.inverse_transform([Y_test])
plt.plot(true_data[0])

## LSTM - Self Activities

 * change the number of features
 * add more neurons to LSTM
 * maybe try adding other layer after LSTM to see what you get
 * was scaling done correct? we scaled before splitting the data
 * was the results of LSTM bettwe or worse than NN? Why?